In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install transformers trl wandb

In [3]:
import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [4]:
reward_df=pd.read_csv("reward_data.csv")
df_train=reward_df[:90]
df_test=reward_df[90:]
df_test=df_test.reset_index()
df1=pd.DataFrame({"text":["<s>[INST] Summarize the question based on the context below. " + "\n[context]: " + df_train['article'][i] + "[\\INST] "+df_train['summary'][i] for i in range(len(df_train))]})
df2=pd.DataFrame({"text":["<s>[INST] Summarize the question based on the context below. " + "\n[context]: " + df_test['article'][i] + "[\\INST] "+df_test['summary'][i] for i in range(len(df_test))]})
df=pd.DataFrame({"text":["<s>[INST] Summarize the question based on the context below. " + "\n[context]: " + reward_df['article'][i] + "[\\INST] "+reward_df['summary'][i] for i in range(len(reward_df))]})
df1['label']=df_train['output']
df2['label']=df_test['output']
df['label']=reward_df['output']
df1['summary']=[df_train['summary'][i] for i in range(len(df_train))]
df2['summary']=[df_test['summary'][i] for i in range(len(df_test))]
df['summary']=[reward_df['summary'][i] for i in range(len(reward_df))]

In [5]:
df_train_for_gen=pd.DataFrame({"text":["<s>[INST] Summarize the question based on the context below. " + "\n[context]: " + df_train['article'][i] + "[\INST] " for i in range(len(df_train)) if len(df_train['article'][i].split(" "))<700]})
df_train_for_gen['summary']=df1['summary']
df_test_for_gen=pd.DataFrame({"text":["<s>[INST] Summarize the question based on the context below. " + "\n[context]: " + df_test['article'][i] + "[\INST] " for i in range(len(df_test)) if len(df_test['article'][i].split(" "))<700]})
df_test_for_gen['summary']=df2['summary']
df_for_gen=pd.DataFrame({"text":["<s>[INST] Summarize the question based on the context below. " + "\n[context]: " + reward_df['article'][i] + "[\INST] " for i in range(len(reward_df)) if len(reward_df['article'][i].split(" "))<700]})
df_for_gen['summary']=df['summary']

In [6]:
from datasets import *
ds3=Dataset.from_pandas(df_for_gen)
ds1=ds3.train_test_split(test_size=0.1)
ds2=DatasetDict({'train':ds1['train'],
                 'test':ds1['test']})
ds2

DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 90
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 10
    })
})

In [7]:
#from datasets import load_dataset

#dataset = load_dataset("HuggingFaceH4/cherry_picked_prompts", split="train")
ds1 = ds2.rename_column("text", "query")
#dataset = dataset.remove_columns(["meta", "completion"])

In [8]:
from trl import PPOConfig

config = PPOConfig(
    model_name="Ritirupa/sft-llama-2-7b-sft",
    learning_rate=1.41e-5,
)

In [9]:
from transformers import AutoTokenizer

from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer

model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [10]:
from transformers import pipeline

reward_model = pipeline("text-classification", model="Ritirupa/gpt2_reward_test",tokenizer="Ritirupa/gpt2_reward_test")

In [11]:
def tokenize(sample):
    sample["input_ids"] = tokenizer.encode(sample["query"])
    return sample

dataset = ds1.map(tokenize, batched=False)

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
from trl import PPOTrainer

ppo_trainer = PPOTrainer(
    model=model,
    config=config,
    dataset=dataset,
    tokenizer=tokenizer,
)

In [ ]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

In [ ]:
from tqdm import tqdm

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]

    #### Get response from SFTModel
    response_tensors = ppo_trainer.generate(query_tensors, **generation_kwargs)
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute reward score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = reward_model(texts)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

#### Save model
ppo_trainer.save_model("my_ppo_model")